In [1]:
import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)

In [2]:
%pwd

'd:\\Projekt\\notebooks'

In [3]:
%cd ..

d:\Projekt


d:\Projekt\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
from src.prep_vid_v2 import get_images_tensor
import pyarrow.parquet as pq
from pathlib import Path
from tqdm import tqdm
from src.model import load_model
import torch
import pandas as pd

d:\Projekt\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Projekt\venv\Lib\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start set

In [5]:
data_path = [Path('data/datasets/eval'), Path('data/datasets/test')]
models = ['adaface', 'arcface']
# models = ['adaface']

In [6]:
for path in data_path:
    for model_name in models:
        dataset = path.parts[-1]
        vids = list(path.rglob('*.mp4'))
        data = {'video_id': [], 'person_id': [], 'embedding': [], 'frame_id': []}
        model = load_model(model_name)
        for video in tqdm(vids, desc=f'Model: {model_name}, Dataset: {dataset}'):
            video_id = video.stem
            person_id = video.parts[-2]
            images = get_images_tensor(video)
            if images is None or images.shape == torch.Size([0]):
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(None)
                data['frame_id'].append(None)
                # print(f'No faces detected in {video}')
                continue
            embeddings = model(images.float().cuda())
            if isinstance(embeddings, torch.Tensor):
                embeddings = embeddings.detach().numpy()
            for i, embedding in enumerate(embeddings):
                embedding = embedding.squeeze()
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(embedding)
                data['frame_id'].append(int(i))
        df = pd.DataFrame(data)
        df.to_parquet(f'data/fusion_win/{model_name}_{dataset}.parquet')



d:\Projekt\models\AdaFace\inference.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  statedict = torch.load(adaface_models[architecture])['state_dict']
Model: adaface, D

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: arcface, Dataset: eval: 100%|██████████| 2487/2487 [55:52<00:00,  1.35s/it]  
d:\Projekt\models\AdaFace\inference.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
 

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: arcface, Dataset: test: 100%|██████████| 2476/2476 [52:55<00:00,  1.28s/it]  


In [7]:
df[(df['video_id'] == '94') & (df['person_id'] == 'id07802')]

,video_id,person_id,embedding,frame_id
6938,94,id07802,"[93.14693, -129.21118, -13.749935, 82.48632, 1...",0.0
6939,94,id07802,"[96.13412, -83.50075, -45.8147, 93.86912, 152....",1.0
6940,94,id07802,"[89.845856, -115.05413, 0.22492495, 75.779564,...",2.0
6941,94,id07802,"[85.505394, -85.0633, -4.0906725, 105.05371, 1...",3.0


In [8]:
path = 'data/datasets/train/'
data_path = [Path(path+str(i)) for i in range(13)]

In [9]:

for path in data_path:
    for model_name in models:
        dataset = path.parts[-2]
        split = path.parts[-1]
        vids = list(path.rglob('*.mp4'))
        data = {'video_id': [], 'person_id': [], 'embedding': [], 'frame_id': []}
        model = load_model(model_name)
        for video in tqdm(vids, desc=f'Model: {model_name}, Dataset: {dataset}'):
            video_id = video.stem
            person_id = video.parts[-2]
            images = get_images_tensor(video)
            if images is None or images.shape == torch.Size([0]):
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(None)
                data['frame_id'].append(None)
                # print(f'No faces detected in {video}')
                continue
            embeddings = model(images.float().cuda())
            if isinstance(embeddings, torch.Tensor):
                embeddings = embeddings.detach().numpy()
            for i, embedding in enumerate(embeddings):
                embedding = embedding.squeeze()
                data['video_id'].append(video_id)
                data['person_id'].append(person_id)
                data['embedding'].append(embedding)
                data['frame_id'].append(int(i))
        df = pd.DataFrame(data)
        df.to_parquet(f'data/fusion_win/{model_name}_{dataset}_{split}.parquet')

d:\Projekt\models\AdaFace\inference.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  statedict = torch.load(adaface_models[architecture])['state_dict']
Model: adaface, D

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: adaface, Dataset: train: 100%|██████████| 4500/4500 [1:16:49<00:00,  1.02s/it]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: adaface, Dataset: train: 100%|██████████| 4273/4273 [1:25:12<00:00,  1.20s/it]  


...

Model: adaface, Dataset: train: 100%|██████████| 4423/4423 [1:27:11<00:00,  1.18s/it]  


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: adaface, Dataset: train: 100%|██████████| 766/766 [18:07<00:00,  1.42s/it]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


Model: arcface, Dataset: train: 100%|██████████| 766/766 [20:37<00:00,  1.62s/it]
